In [1]:
import os

In [2]:
import pandas as pd

In [3]:
import docx

In [4]:
import shutil

In [5]:
import glob

In [6]:
import numpy as np

In [7]:
import sys

In [8]:
import win32com.client

In [9]:
sys.path.append(r'E:\programming\pycomcad\PycomCAD')

In [10]:
import os

In [11]:
from pycomcad import *

In [12]:
def createMN(path):
    filename0=os.path.split(path)[1].split('-')[-1][:5]
    filename='50-F07281Z-'+filename0+'-00.REV.0.docx'
    filename=os.path.join(path,filename)
    shutil.copyfile(r'E:\工作\印尼西加 竣工图\Demo\水工修改说明.docx',filename)
    doc=docx.Document(filename)
    tb=doc.tables[0]
    p=tb.cell(13,10).add_paragraph('    ')
    p.add_run('50-F07281Z-'+filename0+'-00').bold = True
    doc.save(filename)
    print(tb.cell(13,10).text)
    print('新增文档分割线'.center(30,'-'))

In [13]:
def batchCreateMN(path):
    a=0
    for curPath,nexPath,files in os.walk(path):
        if 'REV' in curPath:
            createMN(curPath)
            a+=1
        if nexPath:
            for path in nexPath:
                path=os.path.join(curPath,path)
                batchCreateMN(path)
    return '%s files have been created successfully and correctly' % a

In [14]:
# batchCreateMN(r'E:\工作\印尼西加 竣工图\竣工图')

In [15]:
import sys
sys.path.append(r'E:\programming\pycomcad\PycomCAD')
from pycomcad import *

In [16]:
acad=Autocad()

In [17]:
if not acad.IsEarlyBind:
    acad.TurnOnEarlyBind()

In [18]:
blk=acad.GetEntity() #选择竣工图图标

In [19]:
blk=blk[0]

In [20]:
def is_Chinese(word):
    for ch in word:
        if '\u4e00' <= ch <= '\u9fff':
            return True
    return False

In [21]:
def getSeleCoord(x,y,deltaX,deltaY):
    """
    x,y：左下角选中点的坐标
    deltax，deltay：与右上角点的坐标差，均为正值。
    """
    ini=np.full((4,3),(x,y,0))
    delt=np.array([[-deltaX,deltaY,0],
                   [0,deltaY,0],
                   [0,0,0],
                  [-deltaX,0,0]])
    return (ini+delt).flatten().tolist()

In [24]:
def DealOne(acad,ent,i,jg):
    texts=[]
    data=[None]*11
    data[0]=i+2
    data[2]='xx专业'
    data[5]=3152
    data[6]=2641
    data[9]='0版'
    coords=np.array(ent.Coordinates).reshape((4,2))
    x,y=coords.max(axis=0)[0],coords.min(axis=0)[1]
    print('%s x,y-->'%i,(x,y))
    sltDwNo=acad.GetSelectionSets('DwNo')
    c=getSeleCoord(x-33.35,y+10,51,8.5)
    print('图号范围：',c)
    sltDwNo.SelectByPolygon(Mode=win32com.client.constants.acSelectionSetWindowPolygon,PointsList=VtVertex(*c))
    # 提取，修正，保存图号
    for j in range(sltDwNo.Count):
        dwNo=acad.Handle2Object(sltDwNo.Item(j).Handle)
        if dwNo.EntityName not in ['AcDbMText','AcDbText']:
            continue 
        dwNoStr='50-F07281Z-'+dwNo.TextString[-8:-1]+dwNo.TextString[-1]
        data[1]=dwNoStr
        texts.append((dwNoStr,dwNo.Height,dwNo.InsertionPoint,dwNo.StyleName))
    #提取，修正图名  
    sltDwName=acad.GetSelectionSets('DwName')
    c=getSeleCoord(x-10,y+18.5,86,17)
    print('图名范围',c)
    sltDwName.SelectByPolygon(Mode=win32com.client.constants.acSelectionSetWindowPolygon,PointsList=VtVertex(*c))
    ch=''
    en=''
    for k in range(sltDwName.Count):
        dwName=acad.Handle2Object(sltDwName.Item(k).Handle)
        if dwName.EntityName not in ['AcDbMText','AcDbText']:
            continue
        content=dwName.TextString
        if is_Chinese(content):
            ch=ch+content.replace('施工图','竣工图')
        else:
            en=en+content
        texts.append((content.replace('施工图','竣工图'),dwName.Height,dwName.InsertionPoint,dwName.StyleName))  #存储text信息
    data[3]=ch
    data[4]=en
    
    #选取施工图范围，删除施工图图标
    sltSg=acad.GetSelectionSets('sltSg')
    c=getSeleCoord(x-9,y+9,182,156)
    print('施工图图标范围：',c)
    sltSg.SelectByPolygon(Mode=win32com.client.constants.acSelectionSetWindowPolygon,PointsList=VtVertex(*c))
    for m in range(sltSg.Count):
        ent=sltSg.Item(m)
        ent.Delete()
    print('%s Delete...'% i)
    sltDwNo.Delete()
    sltDwName.Delete()
    sltSg.Delete()
    
    #添加竣工图图标
    jg.Copy()
    jg.Move(Apoint(*jg.InsertionPoint),Apoint(x-10,y+10))
    #添加图名，图号
    for t in texts:
        text=acad.AddText(t[0],Apoint(*t[2]),t[1])
        text.StyleName=t[3]
    return data
    

In [25]:

def batchModify(acad,path,blk):
    """
    移除施工图图标，添加竣工图图标，并读取图纸名称，写入图纸清单
    acad:cad app，需要注意的是需要先打开竣工图图标的图纸
    path:cad路径
    blk:竣工图图标
    """
    dfFilename=path[:-4]+'.xlsx'
    colName=['序号', '图号', '专业', '中文图名', '英文图名', '设计人', '校核人', '计划出处日期', '备注', '版本号', '业主图号']
    acad.OpenFile(path)
    acad.ActivateFile(0)
    jg=acad.DeepClone((blk,),1)[0][0]
    acad.ActivateFile(1)
    slt=acad.GetSelectionSets('slt2')
    slt.SelectOnScreen()               #选择施工的图框
    df=pd.DataFrame(columns=colName,index=range(slt.Count+1)) #存放图号，图名的内容
    df.iloc[0]=[1,None,'xx专业','修改说明','Description of modification',3132,2641,None,None,'0版',None]
    for i in range(slt.Count):
        ent=acad.Handle2Object(slt.Item(i).Handle)
        if ent:
            data=DealOne(acad,ent,i,jg)
            print('data',data)
            df.iloc[i+1]=data
    df.to_excel(dfFilename,index=False)
    slt.Delete()

In [26]:
batchModify(acad,r'E:\工作\印尼西加 竣工图\竣工图\S52循环水系统\S520403循环水泵房上部结构施工图\50-F07281S-S520403 REV.0\S520403循环水泵房上部土建施工图0版.dwg',blk)

0 x,y--> (2399.9395300112155, 142.58980493826857)
图号范围： [2315.5895300112156, 161.08980493826857, 0.0, 2366.5895300112156, 161.08980493826857, 0.0, 2366.5895300112156, 152.58980493826857, 0.0, 2315.5895300112156, 152.58980493826857, 0.0]
图名范围 [2303.9395300112155, 178.08980493826857, 0.0, 2389.9395300112155, 178.08980493826857, 0.0, 2389.9395300112155, 161.08980493826857, 0.0, 2303.9395300112155, 161.08980493826857, 0.0]
施工图图标范围： [2208.9395300112155, 307.58980493826857, 0.0, 2390.9395300112155, 307.58980493826857, 0.0, 2390.9395300112155, 151.58980493826857, 0.0, 2208.9395300112155, 151.58980493826857, 0.0]
0 Delete...
data [2, '50-F07281Z-20403-01', 'xx专业', '', '', 3152, 2641, None, None, '0版', None]
1 x,y--> (3465.747232200316, 142.58980493826857)
图号范围： [3381.397232200316, 161.08980493826857, 0.0, 3432.397232200316, 161.08980493826857, 0.0, 3432.397232200316, 152.58980493826857, 0.0, 3381.397232200316, 152.58980493826857, 0.0]
图名范围 [3369.747232200316, 178.08980493826857, 0.0, 3455.7472

施工图图标范围： [6147.996808179121, -486.966399430843, 0.0, 6329.996808179121, -486.966399430843, 0.0, 6329.996808179121, -642.966399430843, 0.0, 6147.996808179121, -642.966399430843, 0.0]
11 Delete...
data [13, '50-F07281Z-20403-12', 'xx专业', 'GDL4.5-B详图', 'The Detail Drawing of GDL4.5-B', 3152, 2641, None, None, '0版', None]
12 x,y--> (7297.888892149605, -650.1282943664773)
图号范围： [7213.538892149605, -631.6282943664773, 0.0, 7264.538892149605, -631.6282943664773, 0.0, 7264.538892149605, -640.1282943664773, 0.0, 7213.538892149605, -640.1282943664773, 0.0]
图名范围 [7201.888892149605, -614.6282943664773, 0.0, 7287.888892149605, -614.6282943664773, 0.0, 7287.888892149605, -631.6282943664773, 0.0, 7201.888892149605, -631.6282943664773, 0.0]
施工图图标范围： [7106.888892149605, -485.12829436647735, 0.0, 7288.888892149605, -485.12829436647735, 0.0, 7288.888892149605, -641.1282943664773, 0.0, 7106.888892149605, -641.1282943664773, 0.0]
12 Delete...
data [14, '50-F07281Z-20403-13', 'xx专业', '吊车梁安装节点详图', 'Joint Det